In [ ]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf
from snowflake.snowpark.types import StringType
from snowflake.snowpark.types import IntegerType
from config import connection_parameters


In [ ]:
#Definir sesión y objetos Snowflake
sesion = Session.builder.configs(connection_parameters).create()
if sesion != None:
    print("Conectado")
    sesion.use_database('inegi')
    print(sesion.sql("select current_warehouse(), current_database(), current_role()").collect()) 
else:
    print("Error de conexión")

In [ ]:
#Crear vista 
query = "create or replace  view INEGI_DATA as select " + \
 "v:ENTIDAD::int as entidad," + \
 "v:MUN::int as municipio," + \
 "v:NOM_MUN::string as nom_municipio," + \
 "v:LOC::string as localidad," + \
 "v:NOM_LOC::string as nom_localidad," + \
 "v:LONGITUD::float as longitud," + \
 "v:LATITUD::float as latitud," + \
 "v:ALTITUD::int as altitud," + \
 "v:POBTOT::int as pob_total," + \
 "v:POBFEM::int as pob_fem," + \
 "v:POBMAS::int as pob_masc," + \
 "v:PCON_DISC::int as pob_discapacidad," + \
 "v:GRAPROES::int as pob_escolaridad," + \
 "v:GRAPROES_F::int as pob_esco_fem," + \
 "v:GRAPROES_M::int as pob_esco_masc," + \
 "v:PSINDER::int as pob_sssalud," + \
 "v:PDER_SS::int as pob_cssalud," + \
 "v:VIVTOT::int as total_vivienda," + \
 "v:TVIVHAB::int total_habitada," + \
 "v:VPH_INTER::int as hab_internet " + \
 "from INEGI_RAW;"
sesion.sql(query).collect()



In [ ]:
#UDF declaración
entidad_udf = sesion.udf.register_from_file(file_path='@inegi/entidad.py',func_name='nom_entidad',return_type=StringType(),input_types=[IntegerType()],is_permanent=True, name="nom_entidad",stage_location="@inegi")

In [ ]:
#Vista con totales por entidad aplicando UDF para convertir número de entidad x nombre entidad
viewquery = "create or replace view INEGI_MAPA as " + \
"with poblacion_lat as ( select latitud,longitud, nom_entidad(entidad) as nom_entidad," + \
"ROW_NUMBER() OVER(PARTITION BY nom_entidad ORDER BY nom_entidad DESC) AS row_number from INEGI_DATA)," + \
"poblacion_t as (select sum(pob_total) as poblacion_total,nom_entidad(entidad) as nom_entidad from " + \
"INEGI_DATA group by entidad order by poblacion_total desc)" + \
" select pl.nom_entidad,pt.poblacion_total, pl.latitud, pl.longitud" + \
" from poblacion_lat pl left join poblacion_t pt on pl.nom_entidad = pt.nom_entidad" + \
" where row_number = 1;"
sesion.sql(viewquery).collect()

In [ ]:
#Validar la vista solo con totales por entidad
df_entidad = sesion.table("INEGI_MAPA")
df_entidad.show()

In [ ]:
sesion.close()